In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [16]:
orders_features = catalog.load("orders_features")
parameters = catalog.load("parameters")

2022-05-05 18:36:37,753 - kedro.io.data_catalog - INFO - Loading data from `orders_features` (PickleDataSet)...
2022-05-05 18:36:38,500 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...


In [29]:
data[parameters["target_col"]]

0         0
1         1
2         1
3         0
4         0
         ..
531022    1
531023    1
531024    0
531025    0
531026    0
Name: returnShipment, Length: 531027, dtype: int64

In [27]:
data = orders_features['train_test']
for col in data.columns:
    print(col)

weekdayOrdered
weekdayReceived
weDelivery
accOldSys
color
size
userAgeImp
customerID
manufacturerID
salutation
state
monthOrdered
itemID
price
discount
basket_price
basketNArts
durationDelivery
accAgeAtOrder
UserAgeOrder
returnShipment
val_set
budgets
customerIDCatboost
manufacturerIDCatboost
salutationCatboost
stateCatboost
monthOrderedCatboost
itemIDCatboost
10
20
50
100
10_base
20_base
50_base
100_base
10_1_True_lift
10_1_True_conf
10_1_False_lift
10_1_False_conf
10_3_True_lift
10_3_True_conf
10_3_False_lift
10_3_False_conf
10_5_True_lift
10_5_True_conf
10_5_False_lift
10_5_False_conf
10_100_True_lift
10_100_True_conf
10_100_False_lift
10_100_False_conf
20_1_True_lift
20_1_True_conf
20_1_False_lift
20_1_False_conf
20_3_True_lift
20_3_True_conf
20_3_False_lift
20_3_False_conf
20_5_True_lift
20_5_True_conf
20_5_False_lift
20_5_False_conf
20_100_True_lift
20_100_True_conf
20_100_False_lift
20_100_False_conf
50_1_True_lift
50_1_True_conf
50_1_False_lift
50_1_False_conf
50_3_True_lift
50

In [18]:
from sklearn.preprocessing import OrdinalEncoder
def transform_input(data, parameters, test_trans=False, testset_value=1):
    ord_encode =  parameters["coldict"]["ord_encode"]
    if test_trans:
        testset_value +=1
    train = data[data["val_set"]<testset_value]
    test = data[data["val_set"]==testset_value]
    enc = OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value=np.nan) # )
    enc_train = enc.fit_transform(train[ord_encode])
    enc_test = enc.transform(test[ord_encode])
    data.loc[:, ord_encode] = np.nan
    data.loc[train.index, ord_encode] = enc_train
    data.loc[test.index, ord_encode] = enc_test
    for column in ord_encode:
        data[column].fillna(max(data[column]), inplace=True)
    catenc= parameters["coldict"]["cat_pass"] + parameters["coldict"]["cat_catboost"]
    data.loc[:, catenc] = data[catenc].apply(lambda x: pd.to_numeric(x))
    return data

In [21]:
test.index

Int64Index([480949, 480950, 480951, 480952, 480953, 480954, 480955, 480956,
            480957, 480958,
            ...
            531017, 531018, 531019, 531020, 531021, 531022, 531023, 531024,
            531025, 531026],
           dtype='int64', length=50078)

In [24]:
enc.transform(data.loc[test.index, ord_encode])

array([[5.9000e+01, 7.2000e+01, 8.3630e+03, ..., 6.0000e+00, 2.2920e+03,
        0.0000e+00],
       [4.8000e+01, 7.2000e+01, 8.3630e+03, ..., 6.0000e+00, 2.6860e+03,
        0.0000e+00],
       [4.2000e+01, 1.0100e+02, 8.3630e+03, ..., 6.0000e+00, 2.4590e+03,
        1.8000e+01],
       ...,
       [8.3000e+01, 1.1100e+02, 5.8692e+04, ..., 2.0000e+00, 2.4150e+03,
        4.0000e+00],
       [8.3000e+01, 1.0700e+02,        nan, ..., 3.0000e+00, 2.3970e+03,
        4.0000e+00],
       [1.7000e+01, 1.0700e+02,        nan, ..., 3.0000e+00, 2.3970e+03,
        4.0000e+00]])

In [22]:
data = transform_input(data, parameters, test_trans = True)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [10]:
test=False
testset_value=1

In [11]:
ord_encode =  parameters["coldict"]["ord_encode"]
if test:
    testset_value +=1
train = data[data["val_set"]<testset_value]
test = data[data["val_set"]==testset_value]
enc = OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value=np.nan) # )


In [14]:
enc

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=nan)

In [15]:
enc_train = enc.fit_transform(train[ord_encode])
enc_test = enc.transform(test[ord_encode])

In [ ]:
enc_train = enc.fit_transform(train[ord_encode])
enc_test = enc.transform(test[ord_encode])
data.loc[:, ord_encode] = np.nan
data.loc[train.index, ord_encode] = enc_train
data.loc[test.index, ord_encode] = enc_test
for column in ord_encode:
    data[column].fillna(max(data[column]), inplace=True)
catenc= parameters["coldict"]["cat_pass"] + parameters["coldict"]["cat_catboost"]
data.loc[:, catenc] = data[catenc].apply(lambda x: pd.to_numeric(x))
return data

In [6]:
data = transform_input(data, parameters, test = True)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [37]:
results_cols = catalog.load("results_cols")
enc_cols = catalog.load("enc_cols")
res_df = pd.DataFrame(results_cols)
best_cols = []
for ind in res_df.iloc[:-1].index:
    best_cols += [res_df.loc[ind][res_df.loc[ind]==max(res_df.loc[ind])].index.to_series().iloc[0]]
    # print(ind, max(res_df.loc[ind]))

2022-05-03 19:21:24,959 - kedro.io.data_catalog - INFO - Loading data from `results_cols` (PickleDataSet)...
2022-05-03 19:21:24,971 - kedro.io.data_catalog - INFO - Loading data from `enc_cols` (PickleDataSet)...


In [38]:
for col in res_df.columns:
    res_df.loc["item_imp", col] = res_df.loc["feat_imp", col].loc[col] / res_df.loc["feat_imp", col].sum()

In [39]:
res_df.to_csv('/home/ubuntu/dmc/catboost_hyperparams_dmc14.csv', sep=';',  decimal=',')  

In [26]:
res_df.loc[['MCC_test', 'F1_test', 'Recall_test', 'Prec_test']].to_csv('/home/ubuntu/dmc/catboost_hyperparams_dmc14.csv', sep=';')  

In [27]:
res_df.loc["item_imp", :] = 0

In [42]:
res_df.loc["feat_imp", "10_m"]

weekdayOrdered       3.630632
weekdayReceived      7.891083
weDelivery           0.139032
accOldSys            0.155548
color                2.231045
size                 3.092232
userAgeImp           0.054330
customerID          28.126950
manufacturerID       4.582061
salutation           0.220073
state                0.137581
monthOrdered         1.903345
price                1.574509
discount             0.957211
basket_price         6.840658
basketNArts          3.804105
durationDelivery    13.425298
accAgeAtOrder        1.548252
UserAgeOrder         0.644301
10_m                19.041755
dtype: float64

In [33]:
res_df

,itemID,10_m,20_m,50_m,100_m,10_m_smooth,20_m_smooth,50_m_smooth,100_m_smooth,10,...,100_3_False_lift_base,100_3_False_conf_base,100_5_True_lift_base,100_5_True_conf_base,100_5_False_lift_base,100_5_False_conf_base,100_100_True_lift_base,100_100_True_conf_base,100_100_False_lift_base,100_100_False_conf_base
MCC_test,0.350459,0.349533,0.349642,0.351162,0.347027,0.348557,0.347929,0.348522,0.349223,0.334559,...,0.348687,0.349744,0.349312,0.349038,0.348132,0.347543,0.347414,0.349589,0.350012,0.350012
F1_test,0.679209,0.68375,0.682858,0.681856,0.678728,0.681968,0.680713,0.679662,0.682181,0.674401,...,0.683569,0.686444,0.6835,0.685134,0.6825,0.685597,0.685088,0.684884,0.683702,0.683702
Recall_test,0.688042,0.704143,0.700987,0.695913,0.690239,0.699229,0.695753,0.691638,0.699189,0.6898,...,0.704503,0.713253,0.703544,0.709497,0.701506,0.712893,0.711255,0.707979,0.703424,0.703424
Prec_test,0.6706,0.664505,0.665642,0.668355,0.667594,0.665538,0.66631,0.668095,0.665982,0.659674,...,0.663843,0.661577,0.664566,0.662389,0.664497,0.660314,0.660777,0.663248,0.665055,0.665055
MCC_test_multiarts,0.346107,0.346223,0.347073,0.348291,0.343631,0.345627,0.344606,0.345178,0.345277,0.331271,...,0.346755,0.347454,0.346839,0.34585,0.345343,0.345139,0.344813,0.347026,0.347168,0.347168
Recall_test_multiarts,0.711339,0.725555,0.723024,0.718173,0.711972,0.722096,0.718173,0.715304,0.721885,0.709398,...,0.726103,0.734498,0.725977,0.731798,0.723024,0.734666,0.732895,0.729562,0.725344,0.725344
Prec_test_multiarts,0.672771,0.668195,0.669531,0.671863,0.67104,0.668959,0.669617,0.6709,0.668816,0.664336,...,0.668336,0.666029,0.668427,0.66595,0.668487,0.664606,0.664983,0.667374,0.668832,0.668832
feat_imp,weekdayOrdered 3.328506 weekdayReceived ...,weekdayOrdered 3.630632 weekdayReceived ...,weekdayOrdered 3.367893 weekdayReceived ...,weekdayOrdered 3.147455 weekdayReceived ...,weekdayOrdered 3.495504 weekdayReceived ...,weekdayOrdered 3.190424 weekdayReceived ...,weekdayOrdered 3.467760 weekdayReceived ...,weekdayOrdered 3.551326 weekdayReceived ...,weekdayOrdered 3.795206 weekdayReceived ...,weekdayOrdered 0.746436 weekdayReceived ...,...,weekdayOrdered 3.820683 weekdayRece...,weekdayOrdered 4.137883 weekdayRece...,weekdayOrdered 3.008557 weekdayRecei...,weekdayOrdered 3.660580 weekdayRecei...,weekdayOrdered 3.696781 weekdayRece...,weekdayOrdered 3.119039 weekdayRece...,weekdayOrdered 3.963539 weekdayRec...,weekdayOrdered 3.634540 weekdayRec...,weekdayOrdered 4.345198 weekdayRe...,weekdayOrdered 4.345198 weekdayRe...
item_imp,19.880689,19.041755,19.28833,17.6911,17.841714,18.206943,17.19493,15.65874,16.462244,60.550179,...,20.726344,21.097369,22.385395,21.827007,21.116528,22.798296,23.279487,20.694933,19.464575,19.464575


In [8]:
res_df[enc_cols['m_smooth']].iloc[:4]

,10_m_smooth,20_m_smooth,50_m_smooth,100_m_smooth
MCC_test,0.348557,0.347929,0.348522,0.349223
F1_test,0.681968,0.680713,0.679662,0.682181
Recall_test,0.699229,0.695753,0.691638,0.699189
Prec_test,0.665538,0.66631,0.668095,0.665982


In [9]:
res_df[enc_cols['1D']].iloc[:4]

,10,20,50,100
MCC_test,0.334559,0.331253,0.335813,0.331036
F1_test,0.674401,0.674298,0.683493,0.674977
Recall_test,0.6898,0.693076,0.719485,0.695553
Prec_test,0.659674,0.656511,0.650931,0.655583


In [10]:
res_df[enc_cols['1D_base']].iloc[:4]

,10_base,20_base,50_base,100_base
MCC_test,0.349379,0.349401,0.350941,0.347204
F1_test,0.684963,0.682087,0.680628,0.679486
Recall_test,0.708498,0.69867,0.692117,0.692517
Prec_test,0.662941,0.666273,0.669514,0.666936


In [84]:
# new_f["enc"] = "BetaLoo"
# new_f.set_index("enc", append=True, inplace = True)
# mapper = new_f.columns.to_series().str.extract('(\d+)').to_dict()[0]
# new_f.rename(mapper, inplace = True, axis=1)

In [51]:
new_f_b = res_df[enc_cols['1D_base']]
# new_f_b["enc"] = "Beta"
# new_f_b.set_index("enc", append=True, inplace = True)

In [ ]:
[int(s) for s in str.split() if s.isdigit()]

In [52]:
mapper = new_f_b.columns.to_series().str.extract('(\d+)').to_dict()[0]
new_f_b.rename(mapper, inplace = True, axis=1)

<>:1: DeprecationWarning: invalid escape sequence \d
<>:1: DeprecationWarning: invalid escape sequence \d
/tmp/ipykernel_25083/4171924704.py:1: DeprecationWarning: invalid escape sequence \d
  mapper = new_f_b.columns.to_series().str.extract('(\d+)').to_dict()[0]
/tmp/ipykernel_25083/4171924704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_f_b.rename(mapper, inplace = True, axis=1)


In [92]:
pd.concat([, new_f_b.iloc[:4]], keys=['BetaLoo','Beta'])#.swaplevel(0,1,axis=1).sort_index(axis=1)

10        20        50       100        10  \
BetaLoo MCC_test     0.285979  0.287929  0.279059  0.278459       NaN   
        F1_test      0.674225  0.670812  0.665906  0.672716       NaN   
        Recall_test  0.746015  0.730553  0.722921  0.749451       NaN   
        Prec_test     0.61504  0.620104  0.617227  0.610235       NaN   
Beta    MCC_test          NaN       NaN       NaN       NaN  0.303391   
        F1_test           NaN       NaN       NaN       NaN   0.68336   
        Recall_test       NaN       NaN       NaN       NaN  0.760518   
        Prec_test         NaN       NaN       NaN       NaN  0.620417   

                           20        50       100  
BetaLoo MCC_test          NaN       NaN       NaN  
        F1_test           NaN       NaN       NaN  
        Recall_test       NaN       NaN       NaN  
        Prec_test         NaN       NaN       NaN  
Beta    MCC_test     0.299984  0.303987  0.302584  
        F1_test      0.682853  0.685025  0.682709  
        Recall_test  0.762995  0.766791   0.75888  
        Prec_test    0.617946  0.619017  0.620435

In [74]:
pd.concat([new_f.loc["MCC_test"], new_f_b.loc["MCC_test"]], axis=0)

10     0.285979
20     0.287929
50     0.279059
100    0.278459
10     0.303391
20     0.299984
50     0.303987
100    0.302584
Name: MCC_test, dtype: object

In [46]:
# generates double-cols
# https://stackoverflow.com/questions/45307296/join-merge-two-pandas-dataframes-and-use-columns-as-multiindex


## Runs to find best model catboost

In [12]:
models_catboost = catalog.load("models_catboost")

2022-04-30 16:07:23,040 - kedro.io.data_catalog - INFO - Loading data from `models_catboost` (PickleDataSet)...


In [21]:
models_catboost[1]["baseline"].tree_count_

117

In [22]:
models_catboost[1]["mEstimate"].tree_count_

142

In [50]:
models_catboost[1]["BetaLoo2D"].tree_count_

NameError: name 'models_catboost' is not defined

In [6]:
# from hpbandster run
resultdict = catalog.load("resultdict")
resultdict.keys()
# resultdict["baseline"]["id2config"][resultdict["incumbent"]]["config"]

2022-04-30 17:22:59,987 - kedro.io.data_catalog - INFO - Loading data from `resultdict` (PickleDataSet)...


dict_keys(['baseline', 'mEstimate', 'BetaLoo2D'])

In [22]:
resultdict["baseline"]["res"].extract_HBS_learning_curves()

AttributeError: 'Result' object has no attribute 'extract_HBS_learning_curves'

In [25]:
resultdict["baseline"]["res"].get_incumbent_id()

(4, 0, 23)

In [29]:
resultdict["baseline"].keys()#["info"]

dict_keys(['res', 'id2config', 'incumbent', 'all_runs'])

In [42]:
resultdict["baseline"]["all_runs"][0]["config_id"]

(0, 0, 0)

In [45]:
resultdict["baseline"]["all_runs"]

[config_id: (0, 0, 0)	budget: 9.000000	loss: 0.6652376486802705
 time_stamps: 0.006165742874145508 (submitted), 0.007113456726074219 (started), 128.21902918815613 (finished)
 info: {'F1_test': 0.6690903311679282, 'Recall_test': 0.666376478967987, 'Prec_test': 0.6718263783611266, 'MCC_test_multiarts': 0.3320304417405781, 'Recall_test_multiarts': 0.6868348034952582, 'Prec_test_multiarts': 0.673502538071066, 'depth': 6, 'l2_leaf_reg': 28, 'learning_rate': 0.23126286938981494},
 config_id: (0, 0, 0)	budget: 27.000000	loss: 0.6652376486802705
 time_stamps: 1342.4106793403625 (submitted), 1342.4107496738434 (started), 1465.5741028785706 (finished)
 info: {'F1_test': 0.6690903311679282, 'Recall_test': 0.666376478967987, 'Prec_test': 0.6718263783611266, 'MCC_test_multiarts': 0.3320304417405781, 'Recall_test_multiarts': 0.6868348034952582, 'Prec_test_multiarts': 0.673502538071066, 'depth': 6, 'l2_leaf_reg': 28, 'learning_rate': 0.23126286938981494},
 config_id: (0, 0, 0)	budget: 81.000000	loss:

In [49]:
resultdict["baseline"]["res"].get_runs_by_id(resultdict["baseline"]["res"].get_incumbent_id())[-1]["info"]

{'F1_test': 0.671495105778339,
 'Recall_test': 0.6732222705868387,
 'Prec_test': 0.6697767804417148,
 'MCC_test_multiarts': 0.3321804464129841,
 'Recall_test_multiarts': 0.692674038182797,
 'Prec_test_multiarts': 0.6717269076305221,
 'depth': 6,
 'l2_leaf_reg': 28,
 'learning_rate': 0.23697445375162673}

In [8]:
resultdict["baseline"]["id2config"][resultdict["baseline"]["incumbent"]]["config"]

{'depth': 6, 'l2_leaf_reg': 28, 'lr': 0.23697445375162673}

In [9]:
resultdict["mEstimate"]["id2config"][resultdict["baseline"]["incumbent"]]["config"]

{'depth': 10,
 'l2_leaf_reg': 22,
 'lr': 0.1145472944322216,
 'm': 10,
 'smooth': 'm_smooth'}

In [11]:
resultdict["BetaLoo2D"]["id2config"][resultdict["BetaLoo2D"]["incumbent"]]["config"]

{'depth': 8,
 'l2_leaf_reg': 20,
 'lr': 0.04259296073697268,
 'max_basket': 100,
 'n_min': 50,
 'sort': 'lift',
 'weight': False}

In [25]:
resultdict["BetaLoo2D"]["id2config"]

{(0,
  0,
  0): {'config': {'depth': 2,
   'l2_leaf_reg': 3,
   'lr': 0.2411583036197836,
   'max_basket': 1,
   'n_min': 10,
   'sort': 'lift',
   'weight': False}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  1): {'config': {'depth': 4,
   'l2_leaf_reg': 29,
   'lr': 0.2179075482831009,
   'max_basket': 3,
   'n_min': 50,
   'sort': 'conf',
   'weight': True}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  2): {'config': {'depth': 12,
   'l2_leaf_reg': 10,
   'lr': 0.16808606154514735,
   'max_basket': 1,
   'n_min': 10,
   'sort': 'conf',
   'weight': True}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  3): {'config': {'depth': 2,
   'l2_leaf_reg': 17,
   'lr': 0.26712118076578806,
   'max_basket': 5,
   'n_min': 50,
   'sort': 'lift',
   'weight': False}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  4): {'config': {'depth': 4,
   'l2_leaf_reg': 19,
   'lr': 0.11549055858976759,
   'max_basket': 3,
   'n_min': 20,
   'sort': 'lift',
   'w

In [4]:
resultdict.keys()

dict_keys(['baseline', 'mEstimate', 'BetaLoo2D'])

In [5]:
resultdict["baseline"]

{'res': <hpbandster.core.result.Result at 0x7f04ea9388e0>,
 'id2config': {(0,
   0,
   0): {'config': {'depth': 6,
    'l2_leaf_reg': 28,
    'lr': 0.23126286938981494}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   1): {'config': {'depth': 9,
    'l2_leaf_reg': 5,
    'lr': 0.25463626069833356}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   2): {'config': {'depth': 2,
    'l2_leaf_reg': 12,
    'lr': 0.14903219796791203}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   3): {'config': {'depth': 10,
    'l2_leaf_reg': 22,
    'lr': 0.15301090905897227}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   4): {'config': {'depth': 15,
    'l2_leaf_reg': 4,
    'lr': 0.22045669328745487}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   5): {'config': {'depth': 8,
    'l2_leaf_reg': 14,
    'lr': 0.2833705221385834}, 'config_info': {'model_based_pick': False}},
  (0,
   0,
   6): {'config': {'depth': 10,
    'l2_leaf_reg': 14,
    